# Using AutoSDF

Use this file to generate 3D models using AutoSDF
Make sure the AutoSDF repository is located in in the same parent directory as `completiontools-evaluation`

## Importing and set-up

In [1]:
# Import packages and Setup everything
import os
import json
import torch
import pytorch3d.io
import sys
import comptools
from pathlib import Path
# some utility function for visualization from AutoSDF
sys.path.insert(0, '../')
import AutoSDF.utils as utils
from AutoSDF.utils.util_3d import init_mesh_renderer, sdf_to_mesh
from AutoSDF.utils.demo_util import get_shape_comp_opt
from AutoSDF.utils.demo_util import get_shape_comp_model
from AutoSDF.utils.qual_util import save_mesh_as_gif, save_meshes_to_file, get_partial_shape_by_voxels, get_partial_shape_by_range, get_shape_comp_input_mesh
from IPython.display import Image as ipy_image
from IPython.display import display

# enable hot reloading
%reload_ext autoreload
%autoreload 2

# set up the results directory
input_data_folder_path = r"/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3"
voxel_data_folder_path = r"/home/student/Documents/Voxeldata"
res_dir = r"/home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0"
if not os.path.exists(res_dir): os.makedirs(res_dir)

# Define the options, in this case which GPU we want to use
gpu_id = 0
opt = get_shape_comp_opt(gpu_id=gpu_id)
print(opt.device)

# Setup the object completion model
model = get_shape_comp_model(opt, root = "../AutoSDF")    
model.eval()

# Setup the renderer to display gifs
dist, elev, azim = 1.7, 20, 110
mesh_renderer = init_mesh_renderer(image_size=256, dist=dist, elev=elev, azim=azim, device=opt.device)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
student
geomatica-linux-jelle
cuda:0
[*] Enc has Attn at i_level, i_block: 3, 0
Working with z of shape (1, 256, 8, 8, 8) = 131072 dimensions.
[*] Dec has Attn at i_level, i_block: 3, 0
[*] VQVAE: weight successfully load from: ../AutoSDF/saved_ckpt/pvqvae-snet-all-LR1e-4-T0.2-rerun-epoch140.pth
[*] Model has been created: Rand-Transformer-Model
[*] "rand_tf" initialized.
[*] weight successfully load from: ../AutoSDF/saved_ckpt/rand_tf-snet_code-all-LR1e-4-clean-epoch200.pth


In [2]:
input_list = []
for subdir, dirs, files in os.walk(input_data_folder_path):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".pt"):
            #print (filepath)
            input_list.append(filepath)

voxel_list = []
for subdir, dirs, files in os.walk(voxel_data_folder_path):
    for file in files:
        #print os.path.join(subdir, file)
        filepath = subdir + os.sep + file

        if filepath.endswith(".txt"):
            #print (filepath)
            voxel_list.append(filepath)

print(input_list)
print(voxel_list)

['/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_bocht_180/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_T_90/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_bocht_30/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_bocht_60+60/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_verzamel/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_versmal_lang/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_T_45/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_Y_60/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_versmal/sdf.pt', '/home/student/Documents/Data/meshes_data/synthetisch_meshes_v3/buis_recht/recht_S_60/sdf.pt', '/home/student/Doc

## One-click-generation

In [ ]:
for i in range(len(input_list)):
    sdfPath = input_list[i]
    # Get the shape completion input
    print("Getting the shape completion input")
    sdf = torch.load(sdfPath,map_location=torch.device(opt.device))
    fileFolder = Path(input_list[i]).parent.parent.name
    fileName = Path(input_list[i]).parent.name
    print(fileName)
    
    for j in range(len(voxel_list)):
        jsonPath = voxel_list[j]
        jsonName = Path(voxel_list[j]).stem
        print(jsonName)
        occ_grid = comptools.read_voxel_file(jsonPath)
        shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

        # Define the incomplete mesh
        print("perform shape completion")
        input_mesh = get_shape_comp_input_mesh(shape_comp_input['sdf'], shape_comp_input['sdf_missing'])
        input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
        gen_mesh = sdf_to_mesh(comp_sdf)          # completed shape

        # save as gif
        #gen_gif_name = f'{res_dir}/shape-comp-struct-input-gen.gif'
        #save_mesh_as_gif(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name,device = opt.device)
        #display(ipy_image(gen_gif_name))

        # Save to files
        print("Saving Meshes: " + str(i) + ", " + str(j))
        saveFolder =  os.path.join(res_dir,fileFolder, fileName, jsonName)
        if not os.path.exists(saveFolder): os.makedirs(saveFolder)
        save_meshes_to_file(gen_mesh,saveFolder)
print("Done")

In [ ]:
for i in range(len(input_list)):
    sdfPath = input_list[i]
    print("Getting the shape completion input")
    sdf = torch.load(sdfPath, map_location=torch.device(opt.device))

    fileFolder = Path(input_list[i]).parent.parent.name
    fileName = Path(input_list[i]).parent.name
    print(fileName)
    
    for j in range(len(voxel_list)):
        jsonPath = voxel_list[j]
        jsonName = Path(voxel_list[j]).stem
        print(jsonName)
        occ_grid = comptools.read_voxel_file(jsonPath)

        saveFolder = os.path.join(res_dir, fileFolder, fileName, jsonName)
        result_mesh_path = os.path.join(saveFolder, "gen_mesh.ply")  # of .obj, afhankelijk van je outputformaat

        if os.path.exists(result_mesh_path):
            print(f"Resultaat bestaat al voor {fileName} - {jsonName}, overslaan.")
            continue

        shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

        print("perform shape completion")
        input_mesh = get_shape_comp_input_mesh(shape_comp_input['sdf'], shape_comp_input['sdf_missing'])
        input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
        gen_mesh = sdf_to_mesh(comp_sdf)

        print("Saving Meshes: " + str(i) + ", " + str(j))
        if not os.path.exists(saveFolder): os.makedirs(saveFolder)
        save_meshes_to_file(gen_mesh, saveFolder)

print("Done")


In [ ]:
for i in range(len(input_list)):
    sdfPath = input_list[i]
    print("Getting the shape completion input")
    sdf = torch.load(sdfPath, map_location=torch.device(opt.device))
    fileFolder = Path(input_list[i]).parent.parent.name
    fileName = Path(input_list[i]).parent.name
    print(fileName)
    
    for j in range(len(voxel_list)):
        jsonPath = voxel_list[j]
        jsonName = Path(voxel_list[j]).stem
        print(jsonName)
        occ_grid = comptools.read_voxel_file(jsonPath)
        shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

        # perform shape completion
        print("Perform shape completion")
        try:
            input_mesh = get_shape_comp_input_mesh(
                shape_comp_input['sdf'], 
                shape_comp_input['sdf_missing']
            )
            
            if input_mesh is None:
                raise ValueError("input_mesh is None — shape_comp_input may be invalid.")

            input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
            gen_mesh = sdf_to_mesh(comp_sdf)

            if gen_mesh is None:
                raise ValueError("gen_mesh is None — shape completion failed or sdf was empty.")

        except Exception as e:
            print(f"[ERROR] Skipping (i={i}, j={j}) due to: {e}")
            continue  # skip to next loop iteration

        # Save the output
        print("Saving Meshes: " + str(i) + ", " + str(j))
        saveFolder = os.path.join(res_dir, fileFolder, fileName, jsonName)
        os.makedirs(saveFolder, exist_ok=True)
        save_meshes_to_file(gen_mesh, saveFolder)

print("Done")

In [ ]:
for i in range(len(input_list)):
    sdfPath = input_list[i]
    print("Getting the shape completion input")
    sdf = torch.load(sdfPath, map_location=torch.device(opt.device))
    fileFolder = Path(input_list[i]).parent.parent.name
    fileName = Path(input_list[i]).parent.name
    print(fileName)
    
    for j in range(len(voxel_list)):
        jsonPath = voxel_list[j]
        jsonName = Path(voxel_list[j]).stem
        print(jsonName)

        # Bepaal het pad waar de resultaten worden opgeslagen
        saveFolder = os.path.join(res_dir, fileFolder, fileName, jsonName)

        # Als er al een .obj-bestand is, sla deze combinatie over
        if os.path.exists(saveFolder) and any(f.endswith(".obj") for f in os.listdir(saveFolder)):
            print(f"[INFO] Skip (i={i}, j={j}) — results already exist in {saveFolder}")
            continue

        occ_grid = comptools.read_voxel_file(jsonPath)
        shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

        print("Perform shape completion")
        try:
            input_mesh = get_shape_comp_input_mesh(
                shape_comp_input['sdf'], 
                shape_comp_input['sdf_missing']
            )
            
            if input_mesh is None:
                print("input_mesh is None — shape_comp_input may be invalid.")
                continue
                #raise ValueError("input_mesh is None — shape_comp_input may be invalid.")

            input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
            gen_mesh = sdf_to_mesh(comp_sdf)

            if gen_mesh is None:
                print("gen_mesh is None — shape completion failed or sdf was empty.")
                continue
                #raise ValueError("gen_mesh is None — shape completion failed or sdf was empty.")

        except Exception as e:
            print(f"[ERROR] Skipping (i={i}, j={j}) due to: {e}")
            continue  # skip to next loop iteration

        print("Saving Meshes: " + str(i) + ", " + str(j))
        os.makedirs(saveFolder, exist_ok=True)
        save_meshes_to_file(gen_mesh, saveFolder)

print("Done")

Getting the shape completion input
recht_bocht_180
voxelData_schijf_links_4
[INFO] Skip (i=0, j=0) — results already exist in /home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0/buis_recht/recht_bocht_180/voxelData_schijf_links_4
voxelData_schijf_links_5
[INFO] Skip (i=0, j=1) — results already exist in /home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0/buis_recht/recht_bocht_180/voxelData_schijf_links_5
voxelData_schijf_links_2
[INFO] Skip (i=0, j=2) — results already exist in /home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0/buis_recht/recht_bocht_180/voxelData_schijf_links_2
voxelData_schijf_links_6
[INFO] Skip (i=0, j=3) — results already exist in /home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0/buis_recht/recht_bocht_180/voxelData_schijf_links_6
voxelData_schijf_links_3
[INFO] Skip (i=0, j=4) — results already exist in /home/student/Documents/Data/AutoSDF_results/AutoSDF_results_0/buis_recht/recht_bocht_180/voxelData_schijf_links_3

[*] autoregressively inferencing...: 100%|██████████| 128/128 [00:05<00:00, 22.64it/s]


> /home/student/Documents/AutoSDF/utils/qual_util.py(157)get_shape_comp_input_mesh()
    155 
    156     # combine
--> 157     try:
    158         mesh_comb = structures.join_meshes_as_scene([mesh_part, mesh_res])
    159     except:

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Saving Meshes: 32, 2
voxelData_schijf_links_6
Loading the Json file
Perform shape completion


[*] autoregressively inferencing...:  99%|█████████▉| 380/384 [00:11<00:00, 20.08it/s]

# Step-by-step

## Import the SDF

In [ ]:
#sdfPath = "/srvgentjkd98p2/L/Recordings/2015-01 ShapeNetCore/03001627/1a8bbf2994788e2743e99e0cae970928/models/model_normalized_sdf.pt"
sdfPath = r"/home/student/Documents/Data/Input_puntenwolken/Revit_kopie thesis - 3D View - 3D_Brouwerij_buis_rechthoek/sdf.pt"
sdf = torch.load(sdfPath,map_location=torch.device(opt.device))

In [ ]:
gen_sdf_as_mesh = sdf_to_mesh(sdf) 

# save as gif
gif_name = f'{res_dir}/shape-example.gif'
save_mesh_as_gif(mesh_renderer, gen_sdf_as_mesh, nrow=1, out_name=gif_name)
ipy_image(gif_name)

## Get The Partial Range

### By Voxels

In [ ]:
# Load the JSON file
with open('./demo_data/voxelgrid_half.txt', 'r') as file:
    data = json.load(file)
voxels = data['voxels']
occ_grid = []
for voxel in voxels:
    id = voxel['gridIndex']
    occ_grid.append([id['x'], id['y'], id['z']])

shape_comp_input = get_partial_shape_by_voxels(sdf, occ_grid, device=opt.device)

### By Range

In [ ]:
completionPercent = "50"
# range: -1 ~ 1.
# x: left-to-right; y: bottom-to-top; z: front-to-back
min_x, max_x = -1., 1.
min_y, max_y = -1, 1.
min_z, max_z = 0.3, 1.
input_range = {'x1': min_x, 'x2': max_x, 'y1': min_y, 'y2': max_y, 'z1': min_z, 'z2': max_z}

shape_comp_input = get_partial_shape_by_range(sdf, input_range)

### Get Partial Mesh

In [ ]:
input_mesh = get_shape_comp_input_mesh(shape_comp_input['sdf'], shape_comp_input['sdf_missing'])

# save as gif
gif_name = f'{res_dir}/shape-comp-struct-input-inp.gif'
save_mesh_as_gif(mesh_renderer, input_mesh, nrow=1, out_name=gif_name, device = opt.device)
display(ipy_image(gif_name))

## Shape Completion

In [ ]:
""" perform shape completion """
input_mesh, comp_sdf = model.shape_comp(shape_comp_input, bs=3, topk=30)
gen_mesh = sdf_to_mesh(comp_sdf)          # completed shape

# save as gif
gen_gif_name = f'{res_dir}/shape-comp-struct-input-gen.gif'
save_mesh_as_gif(mesh_renderer, gen_mesh, nrow=3, out_name=gen_gif_name,device = opt.device)
display(ipy_image(gen_gif_name))

In [ ]:
save_meshes_to_file(gen_mesh, res_dir)